In [20]:
import numpy as np
import time
import torch
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')

In [22]:
data = np.load('data/processed_data.npz')
X_train_img = data['X_train_img']
y_train = data['y_train']
X_test_img = data['X_test_img']
y_test = data['y_test']

In [23]:
# Load Baseline
with open('results/baseline_metrics.txt', 'r') as f:
    content = f.read().strip().split(',')
    base_acc = float(content[0])
    base_time = float(content[1])

print(f"Baseline (784 dims) - Accuracy: {base_acc:.4f}")

Baseline (784 dims) - Accuracy: 0.9692


In [ ]:
# Model Definition & Loading
class ConvAutoencoder(nn.Module):
    def __init__(self, latent_dim):
        super(ConvAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), nn.ReLU(),
            nn.Flatten(), nn.Linear(32 * 7 * 7, latent_dim), nn.ReLU()
        )
        
        # Decoder
        self.decoder_fc = nn.Linear(latent_dim, 32 * 7 * 7)
        self.decoder_conv = nn.Sequential(
            nn.Unflatten(1, (32, 7, 7)),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1), nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        x_recon = self.decoder_fc(encoded)
        decoded = self.decoder_conv(x_recon)
        return decoded, encoded

BEST_DIM = 32
model = torch.load(f'models/conv_ae_{BEST_DIM}.pth', map_location=device, weights_only=False)
model.eval()
print("Model Loaded.")

Model Loaded.


In [ ]:
# Compress Data (Inference)
print("Compressing dataset...")
start_encode = time.time()

def get_latent_vectors(data_arr, batch_size=512):
    tensor_data = torch.Tensor(data_arr)
    loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(tensor_data), batch_size=batch_size)
    latents = []
    
    with torch.no_grad():
        for batch in loader:
            inputs = batch[0].to(device)
            
            # The forward method returns (decoded, encoded)
            # We only care about 'encoded' (the latent vector)
            _, encoded = model(inputs)
            
            latents.append(encoded.cpu().numpy())
            
    return np.vstack(latents)

X_train_encoded = get_latent_vectors(X_train_img)
X_test_encoded = get_latent_vectors(X_test_img)

print(f"Encoded Shape: {X_train_encoded.shape}")

Compressing dataset...
Encoded Shape: (60000, 32)


In [ ]:
# Train Classifier & Evaluate
print(f"Training Random Forest on PyTorch-Compressed Data...")
start_train = time.time()

clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=42)
clf.fit(X_train_encoded, y_train)

compressed_time = time.time() - start_train
y_pred = clf.predict(X_test_encoded)
compressed_acc = accuracy_score(y_test, y_pred)

# Stats
original_size = 784 * 4
compressed_size = BEST_DIM * 4
reduction_ratio = original_size / compressed_size

print("\n--- PYTORCH RESULTS ---")
print(f"{'Metric':<20} | {'Original':<10} | {'Compressed':<10}")
print("-" * 45)
print(f"{'Accuracy':<20} | {base_acc:.4f}     | {compressed_acc:.4f}")
print(f"{'Training Time (s)':<20} | {base_time:.2f}      | {compressed_time:.2f}")
print(f"{'Dimensions':<20} | {784}        | {BEST_DIM}")
print(f"{'Compression':<20} | 1.0x       | {reduction_ratio:.1f}x")

Training Random Forest on PyTorch-Compressed Data...

--- PYTORCH RESULTS ---
Metric               | Original   | Compressed
---------------------------------------------
Accuracy             | 0.9692     | 0.9560
Training Time (s)    | 3.34      | 2.23
Dimensions           | 784        | 32
Compression          | 1.0x       | 24.5x
